<a href="https://www.kaggle.com/code/ujjawalsingh10/fake-news-classifier-with-bow-lstm?scriptVersionId=120263077" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import re

In [3]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [4]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
#Getting the independent features
X =df.drop(['label', 'id'], axis = 1)

In [6]:
X.head(5)

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
#Dependent Features
y = df['label']

In [8]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [9]:
df.shape

(20800, 5)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [11]:
df.isnull().values.any() #Checking for null values

True

In [12]:
#Dropping the null value examples
df = df.dropna()
df.isnull().values.any()

False

In [13]:
df.shape

(18285, 5)

In [14]:
messages = df.copy()

In [15]:
messages.reset_index(inplace = True) ## To reset the index changed due to removal of null values

In [16]:
len(messages)

18285

In [17]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
#Creating the bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, ngram_range = (1,3))  # SElecting the 5000 most occuring words
X = cv.fit_transform(corpus).toarray()

In [19]:
y = messages['label']

In [20]:
#Splitting the dataset into train and test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

In [21]:
cv.get_feature_names()[:20]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest']

In [22]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [23]:
count_df = pd.DataFrame(X_train, columns = cv.get_feature_names())
count_df[:5]

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,...,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [24]:
### Using MultinomialNB ALgorithm
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [25]:
from sklearn import metrics
import numpy as np
import itertools

In [26]:
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print(f"Accuracy Score: {score:.3f}")

Accuracy Score: 0.902


In [27]:
##Multinomial classifier with hyperparameter
classifier = MultinomialNB(alpha = 0.1)

In [28]:
#Selecting alpha for best performance
previous_score = 0
for alpha in np.arange(0,1,0.1):
    sub_classifier = MultinomialNB(alpha = alpha)
    sub_classifier.fit(X_train, y_train)
    y_pred = sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score > previous_score:
        classifier = sub_classifier
    print(f"Alpha: {alpha}, Score: {score}")

/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


Alpha: 0.0, Score: 0.8903065451532726
Alpha: 0.1, Score: 0.9020712510356255
Alpha: 0.2, Score: 0.9025683512841757
Alpha: 0.30000000000000004, Score: 0.9024026512013256
Alpha: 0.4, Score: 0.9017398508699255
Alpha: 0.5, Score: 0.9015741507870754
Alpha: 0.6000000000000001, Score: 0.9022369511184756
Alpha: 0.7000000000000001, Score: 0.9025683512841757
Alpha: 0.8, Score: 0.9015741507870754
Alpha: 0.9, Score: 0.9017398508699255


In [29]:
#Get features names
feature_names = cv.get_feature_names()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [30]:
## Most real /// The more negative the value the more chances of it being fake
sorted(zip(classifier.coef_[0], feature_names), reverse = True)[:20]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


[(-4.000149156604985, 'trump'),
 (-4.287872694443541, 'hillari'),
 (-4.396389621061519, 'clinton'),
 (-4.899969726208735, 'elect'),
 (-5.176598600897756, 'new'),
 (-5.234730366348767, 'comment'),
 (-5.273968180973631, 'video'),
 (-5.3868167681180115, 'war'),
 (-5.396821854078974, 'us'),
 (-5.412019714988405, 'hillari clinton'),
 (-5.417137433425386, 'fbi'),
 (-5.48068448454208, 'vote'),
 (-5.566255475855405, 'email'),
 (-5.578238842742501, 'world'),
 (-5.634015380199913, 'obama'),
 (-5.734501455772904, 'donald'),
 (-5.763095255139644, 'donald trump'),
 (-5.785090276725191, 'russia'),
 (-5.846224665218559, 'day'),
 (-5.862110622807369, 'america')]

## Fake News Classifier using LSTM

In [31]:
df2 = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [32]:
df2.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
##dropping the Null values
df2 = df.dropna()

In [34]:
# Getting the independent features
X = df2.drop('label', axis = 1)

In [35]:
#Getting the dependent features
y = df2['label']

In [36]:
X.shape, y.shape

((18285, 4), (18285,))

In [37]:
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Dense

In [38]:
##Vocabulary size
voc_size = 5000

## One hot Representation

In [39]:
messages = X.copy()

In [40]:
messages.reset_index(inplace = True)

In [41]:
import nltk
import re
from nltk.corpus import stopwords

In [42]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [43]:
## Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review2 = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review2 = review2.lower()
    review2 = review2.split()
    
    review2 = [ps.stem(word) for word in review2 if word not in stopwords.words('english')]
    review2 = ' '.join(review2)
    corpus.append(review2)

In [44]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

#### One hot representation

In [45]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[:5]

[[3461, 3772, 2211, 220, 4404, 3593, 2286, 2254, 1443, 4823],
 [4851, 4209, 3976, 2848, 4326, 3998, 4881],
 [2431, 611, 4544, 2325],
 [1567, 1616, 2100, 3679, 3147, 2649],
 [1289, 4326, 1827, 3237, 4934, 1081, 4326, 2037, 152, 204]]

#### Embedding representation

In [46]:
#To make the sentences of equal length
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 2254 1443 4823]
 [   0    0    0 ... 4326 3998 4881]
 [   0    0    0 ...  611 4544 2325]
 ...
 [   0    0    0 ... 4457 3032 4962]
 [   0    0    0 ... 3480 1379 2811]
 [   0    0    0 ... 3950 1355 2902]]


In [47]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3461,
       3772, 2211,  220, 4404, 3593, 2286, 2254, 1443, 4823], dtype=int32)

In [48]:
## Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [50]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

### Model Training

In [52]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 11s 44ms/step - loss: 0.3301 - accuracy: 0.8425 - val_loss: 0.2038 - val_accuracy: 0.9120
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1340 - accuracy: 0.9471 - val_loss: 0.2107 - val_accuracy: 0.9082
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0922 - accuracy: 0.9660 - val_loss: 0.2183 - val_accuracy: 0.9171
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0676 - accuracy: 0.9770 - val_loss: 0.2915 - val_accuracy: 0.9173
Epoch 5/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0501 - accuracy: 0.9845 - val_loss: 0.2593 - val_accuracy: 0.9140
Epoch 6/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0314 - accuracy: 0.9904 - val_loss: 0.3559 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0244 - accuracy: 0.9928 - val_loss: 0.3685 - val_accuracy: 0.908

#### Adding a dropout

In [53]:
from tensorflow.keras.layers import Dropout
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

### Performance metrics and accuracy

In [54]:
pred = model.predict(X_test)
y_pred=np.argmax(pred,axis=1)

In [55]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [56]:
accuracy_score(y_test, y_pred)

0.5665285832642917

In [57]:
confusion_matrix(y_test, y_pred)

array([[3419,    0],
       [2616,    0]])